# Tutorial 06, case 3a: advection diffusion reaction control problem with Neumann control

In this tutorial we solve the optimal control problem

$$\min J(y, u) = \frac{1}{2} \int_{\Omega} (y - y_d)^2 dx + \frac{\alpha}{2} \int_{\Gamma_2} u^2 ds$$
s.t.
$$\begin{cases}
- \epsilon \Delta y + \beta \cdot \nabla y + \sigma y = f      & \text{in } \Omega\\
                                \epsilon \partial_n y = 0      & \text{on } \Gamma_1\\
                                \epsilon \partial_n y = u      & \text{on } \Gamma_2\\
                                \epsilon \partial_n y = 0      & \text{on } \Gamma_3\\
                                                    y = 0      & \text{on } \Gamma_4
\end{cases}$$

where
$$\begin{align*}
& \Omega               & \text{unit square}\\
& \Gamma_1             & \text{bottom boundary of the square}\\
& \Gamma_2             & \text{left boundary of the square}\\
& \Gamma_3             & \text{top boundary of the square}\\
& \Gamma_4             & \text{right boundary of the square}\\
& u \in L^2(\Gamma_2)  & \text{control variable}\\
& y \in H^1(\Omega)    & \text{state variable}\\
& \alpha > 0           & \text{penalization parameter}\\
& y_d                  & \text{desired state}\\
& f                    & \text{forcing term}
\end{align*}$$
using an adjoint formulation solved by a one shot approach

In [ ]:
import numpy as np
from petsc4py import PETSc
from ufl import as_vector, grad, inner, Measure, replace, TestFunction, TrialFunction
from dolfinx import Constant, DirichletBC, Function, FunctionSpace, MPI
from dolfinx.cpp.la import create_petsc_index_sets, GhostBlockLayout, VecSubVectorReadWrapper
from dolfinx.cpp.mesh import GhostMode
from dolfinx.fem import (apply_lifting, assemble_matrix, assemble_matrix_block, assemble_scalar,
                         assemble_vector, assemble_vector_block, create_vector_block, DofMapRestriction,
                         locate_dofs_topological, set_bc)
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Mesh

In [ ]:
mesh = XDMFFile(MPI.comm_world, "data/square.xdmf").read_mesh(GhostMode.none)
subdomains = XDMFFile(MPI.comm_world, "data/square_subdomains.xdmf").read_mf_size_t(mesh)
boundaries = XDMFFile(MPI.comm_world, "data/square_boundaries.xdmf").read_mf_size_t(mesh)
boundaries_2 = np.where(boundaries.values == 2)[0]
boundaries_4 = np.where(boundaries.values == 4)[0]

In [ ]:
# Define associated measures
dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

### Function spaces

In [ ]:
Y = FunctionSpace(mesh, ("Lagrange", 2))
U = FunctionSpace(mesh, ("Lagrange", 2))
Q = Y.clone()

### Restrictions

In [ ]:
dofs_Y = np.arange(0, Y.dofmap.index_map.block_size * (
    Y.dofmap.index_map.size_local + Y.dofmap.index_map.num_ghosts))
dofs_U = locate_dofs_topological(U, boundaries.dim, boundaries_2)
dofs_Q = dofs_Y
restriction_Y = DofMapRestriction(Y.dofmap, dofs_Y)
restriction_U = DofMapRestriction(U.dofmap, dofs_U)
restriction_Q = DofMapRestriction(Q.dofmap, dofs_Q)
restriction = [restriction_Y, restriction_U, restriction_Q]

### Trial and test functions

In [ ]:
(y, u, p) = (TrialFunction(Y), TrialFunction(U), TrialFunction(Q))
(z, v, q) = (TestFunction(Y), TestFunction(U), TestFunction(Q))

 ### Problem data

In [ ]:
alpha = 1.e-5
y_d = 1.
epsilon = 1.e-1
beta = as_vector((-1., -2.))
sigma = 1.
ff = 1.
bc0 = Function(Y)

### Optimality conditions

In [ ]:
zero = Constant(mesh, 0.)
state_operator = (epsilon * inner(grad(y), grad(q)) * dx + inner(beta, grad(y)) * q * dx
                  + sigma * y * q * dx)
adjoint_operator = (epsilon * inner(grad(p), grad(z)) * dx - inner(beta, grad(p)) * z * dx
                    + sigma * p * z * dx)
a = [[y * z * dx, None, adjoint_operator],
     [None, alpha * u * v * ds(2), - p * v * ds(2)],
     [state_operator, - u * q * ds(2), zero * p * q * dx]]
f = [y_d * z * dx,
     zero * v * dx,
     ff * q * dx]
bdofs_Y_4 = locate_dofs_topological((Y, Y), mesh.topology.dim - 1, boundaries_4)
bdofs_Q_4 = locate_dofs_topological((Q, Y), mesh.topology.dim - 1, boundaries_4)
bc = [DirichletBC(bc0, bdofs_Y_4, Y),
      DirichletBC(bc0, bdofs_Q_4, Q)]

### Solution

In [ ]:
(y, u, p) = (Function(Y), Function(U), Function(Q))

### Cost functional

In [ ]:
J = 0.5 * inner(y - y_d, y - y_d) * dx + 0.5 * alpha * inner(u, u) * ds(2)

### Uncontrolled functional value

In [ ]:
# Extract state forms from the optimality conditions
a_state = replace(a[2][0], {q: z})
f_state = replace(f[2], {q: z})
bc_state = [bc[0]]

In [ ]:
# Assemble the linear system for the state
A_state = assemble_matrix(a_state, bcs=bc_state)
A_state.assemble()
F_state = assemble_vector(f_state)
apply_lifting(F_state, [a_state], [bc_state])
F_state.ghostUpdate(addv=PETSc.InsertMode.ADD, mode=PETSc.ScatterMode.REVERSE)
set_bc(F_state, bc_state)

In [ ]:
# Solve
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A_state)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F_state, y.vector)
y.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
J_uncontrolled = MPI.sum(mesh.mpi_comm(), assemble_scalar(J))
print("Uncontrolled J =", J_uncontrolled)
assert np.isclose(J_uncontrolled, 0.23058801)

In [ ]:
plot(y, title="uncontrolled state")

### Optimal control

In [ ]:
# Assemble the block linear system for the optimality conditions
A = assemble_matrix_block(a, bcs=bc, restriction=(restriction, restriction))
A.assemble()
F = assemble_vector_block(f, a, bcs=bc, restriction=restriction)

In [ ]:
# Solve
yup = create_vector_block(f, restriction=restriction)
ksp = PETSc.KSP()
ksp.create(mesh.mpi_comm())
ksp.setOperators(A)
ksp.setType("preonly")
ksp.getPC().setType("lu")
ksp.getPC().setFactorSolverType("mumps")
ksp.setFromOptions()
ksp.solve(F, yup)

In [ ]:
# Split the block solution in components
index_sets = create_petsc_index_sets(
    [Y.dofmap.index_map, U.dofmap.index_map, Q.dofmap.index_map],
    ghost_block_layout=GhostBlockLayout.trailing)
restricted_index_sets = create_petsc_index_sets(
    [restriction_.index_map for restriction_ in restriction],
    ghost_block_layout=GhostBlockLayout.trailing)
unrestricted_to_restricted = [restriction_.unrestricted_to_restricted for restriction_ in restriction]
for i, sub in enumerate((y, u, p)):
    with sub.vector.localForm() as sub_local:
        sub_local[:] = VecSubVectorReadWrapper(yup, index_sets[i], restricted_index_sets[i],
                                               unrestricted_to_restricted[i]).content
    sub.vector.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)

In [ ]:
J_controlled = MPI.sum(mesh.mpi_comm(), assemble_scalar(J))
print("Optimal J =", J_controlled)
assert np.isclose(J_controlled, 0.21176010)

In [ ]:
plot(y, title="state")

In [ ]:
plot(u, title="control")

In [ ]:
plot(p, title="adjoint")